In [2]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode
import nba_api
from nba_api.stats.endpoints import commonplayerinfo, teamgamelog, boxscoresummaryv2
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from pyspark import SparkContext
from pyspark.sql import SQLContext
import discord

In [72]:
nba_teams = teams.get_teams()
df = pd.DataFrame(nba_teams)
team_id = df.id.unique()
print(df)

            id               full_name abbreviation       nickname  \
0   1610612737           Atlanta Hawks          ATL          Hawks   
1   1610612738          Boston Celtics          BOS        Celtics   
2   1610612739     Cleveland Cavaliers          CLE      Cavaliers   
3   1610612740    New Orleans Pelicans          NOP       Pelicans   
4   1610612741           Chicago Bulls          CHI          Bulls   
5   1610612742        Dallas Mavericks          DAL      Mavericks   
6   1610612743          Denver Nuggets          DEN        Nuggets   
7   1610612744   Golden State Warriors          GSW       Warriors   
8   1610612745         Houston Rockets          HOU        Rockets   
9   1610612746    Los Angeles Clippers          LAC       Clippers   
10  1610612747      Los Angeles Lakers          LAL         Lakers   
11  1610612748              Miami Heat          MIA           Heat   
12  1610612749         Milwaukee Bucks          MIL          Bucks   
13  1610612750  Minn

In [118]:
try:
    leaguegamefinder.LeagueGameFinder(player_id_nullable = '2544')
except Exception as e:
    raise('API Timeout Error')

print('API Connection Successful')
#gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = team_id[0]) #API Calling

API Connection Successful


In [119]:
game_df = gamefinder.get_data_frames()[0]
game_df.head(5)
# game_df = game_df_enhanced[["SEASON_ID",'TEAM_ID','GAME_ID','GAME_DATE','MATCHUP','WL','PTS','FGM','FGA','FG_PCT',
#                            'FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PLUS_MINUS']]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42019,1610612751,BKN,Brooklyn Nets,0041900114,2020-08-23,BKN vs. TOR,L,241,122,...,0.758,12,32,44,21,5,6,14,17,-28.0
1,42019,1610612751,BKN,Brooklyn Nets,0041900113,2020-08-21,BKN vs. TOR,L,239,92,...,0.727,9,33,42,24,8,4,13,14,-25.0
2,42019,1610612751,BKN,Brooklyn Nets,0041900112,2020-08-19,BKN @ TOR,L,240,99,...,0.704,7,39,46,25,3,6,17,25,-5.0
3,42019,1610612751,BKN,Brooklyn Nets,0041900111,2020-08-17,BKN @ TOR,L,238,110,...,0.688,9,36,45,33,4,2,14,29,-24.0
4,22019,1610612751,BKN,Brooklyn Nets,0021901309,2020-08-13,BKN vs. POR,L,239,133,...,0.684,12,34,46,28,5,5,14,23,-1.0


In [135]:
#Initialize dataframe
game_df_enhanced = pd.DataFrame(columns = ['SEASON_ID','TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME','GAME_ID','GAME_DATE','MATCHUP','WL','MIN','PTS','FGM','FGA','FG_PCT',
                            'FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PLUS_MINUS'])


In [140]:
#Split team_id into two separate 
team_id_1 = team_id[:15]
team_id_2 = team_id[15:]

#Get game data for all the teams
#Starting with first batch of data
try: 
    for i in range(len(team_id_1)):
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = team_id_1[i])
        temp_df = gamefinder.get_data_frames()[0]
        game_df_enhanced = game_df_enhanced.append(temp_df, sort=False)
except Exception as e:
    raise('API Timeout Error Part 1')

#Second batch of data
if game_df_enhanced.shape[0] > 0:

    try: 
        for i in range(len(team_id_2)):
            gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = team_id_2[i])
            temp_df = gamefinder.get_data_frames()[0]
            game_df_enhanced = game_df_enhanced.append(temp_df, sort=False)
    except Exception as e:
        raise('API Timeout Error Part 2')

else:
    print('Data missing from first batch')


print('Team data successfully loaded into DataFrame')

Team data successfully loaded into DataFrame


In [143]:
print(game_df_enhanced.shape[0])
game_df_enhanced.tail(5)

142085


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
2613,21988,1610612766,CHH,Charlotte Hornets,0028800062,1988-11-12,CHH @ ATL,L,238,111,...,0.813,14,21,35,30,12.0,2,19,26,NaN
2614,21988,1610612766,CHH,Charlotte Hornets,0028800052,1988-11-11,CHH @ WAS,L,240,87,...,0.760,11,32,43,22,9.0,1,23,26,NaN
2615,21988,1610612766,CHH,Charlotte Hornets,0028800024,1988-11-08,CHH vs. LAC,W,240,117,...,0.738,17,38,55,28,9.0,1,17,31,NaN
2616,21988,1610612766,CHH,Charlotte Hornets,0028800015,1988-11-05,CHH @ DET,L,240,85,...,1.000,21,19,40,18,8.0,6,11,21,NaN
2617,21988,1610612766,CHH,Charlotte Hornets,0028800008,1988-11-04,CHH vs. CLE,L,240,93,...,0.650,7,24,31,27,8.0,1,24,25,NaN


In [149]:
#Self join on game_id to avoid duplicates
# games = game_df_enhanced.join(game_df_enhanced, how='inner', lsuffix='GAME_ID', rsuffix='GAME_ID')
games = pd.merge(game_df_enhanced,game_df_enhanced, suffixes = ['_A','_B'], on=['GAME_ID'])

In [150]:
games.head(5)

,SEASON_ID_A,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE_A,MATCHUP_A,WL_A,MIN_A,PTS_A,...,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
0,22019,1610612737,ATL,Atlanta Hawks,0021900969,2020-03-11,ATL vs. NYK,L,265,131,...,0.808,15,38,53,26,6.0,3,17,25,0.0
1,22019,1610612737,ATL,Atlanta Hawks,0021900969,2020-03-11,ATL vs. NYK,L,265,131,...,0.808,15,38,53,26,6.0,3,17,25,0.0
2,22019,1610612737,ATL,Atlanta Hawks,0021900969,2020-03-11,ATL vs. NYK,L,265,131,...,0.735,6,41,47,32,14.0,9,12,23,0.0
3,22019,1610612737,ATL,Atlanta Hawks,0021900969,2020-03-11,ATL vs. NYK,L,265,131,...,0.808,15,38,53,26,6.0,3,17,25,0.0
4,22019,1610612737,ATL,Atlanta Hawks,0021900969,2020-03-11,ATL vs. NYK,L,265,131,...,0.808,15,38,53,26,6.0,3,17,25,0.0
